### IMPORTS

In [174]:
import json
import time

In [35]:
from web3 import Web3

In [16]:
print('[X] Booting up...')

[X] Booting up...


### LOAD DATA FROM EXTERNAL FILES

In [17]:
with open('config/settings.json') as json_file:
    settings = json.load(json_file)

In [18]:
with open('config/identifier.json') as json_file:
    device_info = json.load(json_file)

### CONNECT TO CHAIN VIA WEBSOCKET

In [21]:
web3 = Web3(Web3.WebsocketProvider('ws://' + settings['gateways']['blockchain']['host'] + ':' + str(settings['gateways']['blockchain']['port'])))

In [23]:
if web3.isConnected():
    print('[X] Connected with gateway!')
else:
    print('[ ] Could not connect to gateway!')
    exit

[X] Connected with gateway!


### FETCH WHISPER API

In [46]:
shh = web3.geth.shh

### GENERATE WHISPER KEYPAIR

In [43]:
whisper_id = shh.newKeyPair()

In [47]:
print('[X] Generated keypair!')

[X] Generated keypair!


### SUBSCRIBE TO UPDATE EVENTS

In [48]:
print('[X] Listening for messages...\n')

[X] Listening for messages...



In [62]:
message_event = shh.newMessageFilter({
    'topic': web3.toHex(text='eyya'),
    'symKeyID': settings['whisper']['network']
})

In [ ]:
while(True):
    for event in shh.getMessages(message_event):
    
        # CONVERT MESSAGE PARAMS
        author = web3.toHex(val)
        nickname = author[0:4] + '...' + author[-4:]
        message = web3.toText(event['payload'])
        
        # CHECK THE FIRST WORD FOR TRIGGER
        first = message.split(' ')[0].lower()
        
        # IF A REQUEST IS FOUND
        if (first == '!request'):
            
            # NUKE GARBAGE & SPLIT OUT INDIVIDUAL PARAMS
            checks = message[8:].replace(' ', '').split('&')
            
            # CHECK RESULTS
            results = []
            
            # LOOP THROUGH
            for pair in checks:
    
                # DECONSTRUCT KEY/VALUE
                key, value = pair.split('=')
            
                # MAKE SURE THE KEY EXISTS
                if (key in device_info):
            
                    # CHECK & PUSH PARAM COMPARISON RESULT
                    result = device_info[key] == value
                    results.append(result)
                
                # OTHERWISE, DEFAULT TO BAD RESULT
                else:
                    results.append(False)
                
            # IF ALL THE CHECKS MATCH
            if (results.count(False) == 0):
                
                # RESPONSE TEXT
                response = '@' + nickname + ' - I am available!'
                
                # SLEEP FOR 2 SECONDS
                time.sleep(2)
                
                # RESPOND TO REQUEST
                shh.post({
                    'symKeyID': settings['whisper']['network'],
                    'payload': web3.toHex(text=response),
                    'topic': web3.toHex(text='eyya'),
                    'sig': whisper_id,
                    'powTarget': 2.5,
                    'powTime': 2
                })